# Running FFTs on the GPU in Matlab

This notebook is intended to demonstrate how one could use the GPU for doing Fast Fourier Transforms (FFTs) from Matlab.

There are actually several different ways to accomplish this.

You could use mexfiles to directly interface with the highly-optimized cuFFT, a GPU library for FFTs created by Nvidia.
It is also possible to use Matlab's 'GPU Coder' to interface with GPU libraries and write mexfiles that are compiled using Nvidia's compiler 'nvcc'. However, we will take a closer look at one of the easier ways of using GPUs in Matlab, which is through the parallel processing toolbox.


## Using Jupyter Notebooks

The page you're looking at right now is called a Jupyter Notebook, which is implements a concept called *literate programming*. The idea is that you can interleave documentation, code, and output in one single document. Which makes it also a great tool for creating hands-on tutorials such as this one!

The most important thing to remember is that the notebook is organized as a list of cells. You can execute a cell, by selecting it and pressing **shift+enter**. The output should appear directly below the cell. Let's give it a try so we can see that everything is working!

In [ ]:
disp('Yes, everything is working!')

## GPU Computing in Matlab

Let's start with exploring the functionality that Matlab offers that we will use in this tutorial.

We will use the Matlab data structure called "gpuArray". The idea is that data created using this specialized object remains in the memory of the current GPU device. The gpuArray supports a relatively large list of numeric functions that can be performed on the data by the GPU, including FFTs and matrix multiplications. To transfer the data back from GPU memory to the host memory you have to use the function "gather()".

Before we start with using gpuArray, let's a have look at how many and what kind of GPUs are available in our system. Execute the following cell.

In [ ]:
%print the number of GPUs in the system and display information about each GPU
n = gpuDeviceCount();
disp(n);
for x = 1:n
    disp(gpuDevice(x));
end

After you've executed the previous cell, the output should appear right above this text. You can see that first the number of GPUs present in the system was printed using gpuDeviceCount(), after which we used gpuDevice() to display information about the various GPUs in the system.

Before we continue, we should select a GPU in the system to use for the rest of this notebook. Assuming the GPUs in your system are all equal, you can execute the following cell. Otherwise replace it with just D = gpuDevice(x) where x is the index of the best performing GPU in your system.

In [ ]:
%randomly select a GPU
x = randi(n,1,1);
D = gpuDevice(x);

Now that we have selected a GPU to run our experiments on, we can take a look at the data structure that we will use, called 'gpuArray'

In [ ]:
help gpuArray

The gpuArray class supports a rather extensive list of functions, which is something very important. 

The functions supported by gpuArray will have to be the building blocks for your GPU applications, as long as you want to avoid writing your own GPU kernels and avoid having to transfer data back and forth between Matlab and the GPU. We want to minimize the number of transfers between the GPU's memory and Matlab's memory, because transferring data is expensive in terms of time and energy.

Execute the cell below to list the methods supported by gpuArray.

In [ ]:
methods(gpuArray)

## Fast Fourier Transforms on the GPU in Matlab

Fourier Transforms are among the most important and widely used operations in computational physics and general signal processing. The FFT is a divide-and-conquer algorithm for efficiently computing discrete Fourier transforms of complex or real-valued data sets. Nvidia's cuFFT library is designed and optimized to provide extremely high performance FFTs on GPUs. cuFFT is used internally by Matlab when we apply the ``fft()`` function to a gpuArray.

Let's start with generating some input data, by executing the following cell. We create some noisy signal with two main frequencies hidden in the signal. Read the code below carefully and try to identify which operations will be executed on the GPU.

In [ ]:
% Setup a sample frequency and number of samples
sampleFreq = 1000;
sampleTime = 1/sampleFreq;
numSamples = 2^25;

% Create a gpuArray that will store the empty data structure in GPU memory
timeVec = gpuArray((0:numSamples-1) * sampleTime);

% Create a signal with harmonic components at frequencies 50 and 120 Hz, and add some noise
freq1   = 2 * pi * 50;
freq2   = 2 * pi * 120;
signal  = sin( freq1 .* timeVec ) + sin( freq2 .* timeVec );
signal  = signal + 2 * randn( size( timeVec ), 'gpuArray');

plot( signal(1:1000) );
title( 'Noisy time-domain signal' );

In the code above we compute `timeVec` and copy it to GPU memory. The operations that use timeVec, such as the creation of our signal, will all happen on the GPU. Even `randn()` takes ``'gpuArray'`` as argument, which means that the random numbers will be generated by the GPU and stored in GPU memory.

From our noisy signal it can be rather difficult to identify the frequency components. Therefore, we apply a discrete Fourier transform (using the FFT function) and look at the Power Spectral Density (PSD) to reveal the power of signals at various frequencies. Because the data is stored as a gpuArray, and fft() is one of the supported methods of gpuArray, the FFT will be computed on the GPU.

In [ ]:
% Perform FFT on the GPU
transformedSignal = fft( signal );

% Compute the Power Spectral Density
powerSpectrum = transformedSignal .* conj(transformedSignal) ./ numSamples;

% Display the Power Spectral Density
frequencyVector = sampleFreq/2 * linspace( 0, 1, numSamples/2 + 1 );
plot( frequencyVector, real(powerSpectrum(1:numSamples/2+1)) );
title( 'Power spectral density' );
xlabel( 'Frequency (Hz)' );

The above PSD should show peaks exactly at the frequencies that we've inserted into the data. Now that we know that the GPU FFT function produces sensible output, it is interesting to see what we've gained by using the GPU. After all, the entire reason we're using GPUs is because we want high performance. Let's run the code again, without the plots, and with measuring time.

In [ ]:
clearvars -except D

% Setup a sample frequency and number of samples
sampleFreq = 1000;
sampleTime = 1/sampleFreq;
numSamples = 2^25;

tic
% Create a gpuArray that will store the empty data structure in GPU memory
timeVec = gpuArray((0:numSamples-1) * sampleTime);

% Create a signal with harmonic components at frequencies 50 and 120 Hz, and add some noise
freq1   = 2 * pi * 50;
freq2   = 2 * pi * 120;
signal  = sin( freq1 .* timeVec ) + sin( freq2 .* timeVec );
signal  = signal + 2 * randn( size( timeVec ), 'gpuArray');

% Perform FFT on the GPU
transformedSignal = fft( signal );

% Compute the Power Spectral Density
powerSpectrum = transformedSignal .* conj(transformedSignal) ./ numSamples;

% Display the Power Spectral Density
frequencyVector = sampleFreq/2 * linspace( 0, 1, numSamples/2 + 1 );

toc

And now let's do that again, but without using gpuArray at all:

In [ ]:
clearvars -except D

% Setup a sample frequency and number of samples
sampleFreq = 1000;
sampleTime = 1/sampleFreq;
numSamples = 2^25;

tic
% Create a gpuArray that will store the empty data structure in GPU memory
timeVec = (0:numSamples-1) * sampleTime;

% Create a signal with harmonic components at frequencies 50 and 120 Hz, and add some noise
freq1   = 2 * pi * 50;
freq2   = 2 * pi * 120;
signal  = sin( freq1 .* timeVec ) + sin( freq2 .* timeVec );
signal  = signal + 2 * randn( size( timeVec ));

% Perform FFT on the GPU
transformedSignal = fft( signal );

% Compute the Power Spectral Density
powerSpectrum = transformedSignal .* conj(transformedSignal) ./ numSamples;

% Display the Power Spectral Density
frequencyVector = sampleFreq/2 * linspace( 0, 1, numSamples/2 + 1 );

toc

# Performance analysis

While it's great to see that we've managed to accelerate our Matlab program using the GPU, it is a bit hard to understand what exactly is going on under the hood. We have just put the 'gpuArray' somewhere in our code and suddenly things went faster, but how fast is it? Could we have done things differently to make our code even faster?

To answer these questions, we'll look at how we can analyze the GPU performance of our program. For this we use the Nvidia Visual Profiler, called **nvvp**.

To use nvvp with with Matlab you'll need to know a few tricks. We can start nvvp by simply typing 'nvvp' in the command line on our GPU server. Click the top left button to create a "new session".

You are be prompted with a dialog box in which you have to enter the following:  
Executable: matlab  
Working directory: /home/your-username/gpu-course/matlab_exercises/  
Arguments: -nodisplay -nosplash -nodesktop -r "run('fft_example.m');exit;"  

The above instructs nvvp to call Matlab with our script fft_example.m, but without starting the Matlab IDE.

You should also take a look at the contents of fft_example.m. What you'll see is that the script actually performs the work twice. The reason for this is that calling the GPU for the first time comes with quite a bit of startup overhead in Matlab. To create a cleaner picture of the performance of our program we run the code again after pausing for a few seconds. In a larger piece of software there is no need for this as you will likely call the GPU many times and all but the first few functions should return representative results. 

You are now ready to start profiling our fft_example.m script. You can look at the right-most output of the timeline (separated by the pause of a few seconds we inserted in the script for this reason) to see a few calls to the GPU. If you zoom in it should look something like this:

![nvvp_output](nvvp_screenshot_matlab_fft.png)

We can learn a bit more from this output about the performance of our program. First of all, we can see `Memcpy HtoD` on the left. This is the function that transfers data from the host to device memory, which corresponds to the line:

```
timeVec = gpuArray((0:numSamples-1) * sampleTime);
```

The first kernel executed on the GPU has a very long name `_ZN10gpuruntime...`, which is actually the kernel that is responsible for generating random numbers. It's very interesting to see that Matlab has decided to execute that kernel before the others, while it actually comes as one of the last lines in our script before we call the FFT function.

The next three kernels are kernels generated by Matlab that perform the computations in our script that correspond to the following lines of Matlab code:

```
freq1   = 2 * pi * 50;
freq2   = 2 * pi * 120;
signal  = sin( freq1 .* timeVec ) + sin( freq2 .* timeVec );
signal  = signal + 2 * randn( size( timeVec ), 'gpuArray');
```

After this we see that a bunch of kernels are called that all start with `dpRadix`. These are kernels from the cuFFT library that together perform the FFT on our signal. After that, however, the profiling output is missing from Visual Profiler. The most likely explanation is that the rest of the commands are called by another thread, which is outside of the scope of the profiler. 